In [1]:
import os
import pandas as pd
import json
import numpy as np

# Define paths
electricity_data = r".\raw\electricity_raw_data"
weather_data = r".\raw\weather_raw_data"
output_dir = r".\merged_output"
output_file = os.path.join(output_dir, "merged.csv")

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

def load_electricity_data(folder_path):
    electricity_data = []
    for file in os.listdir(folder_path):
        if file.endswith(".json"):
            with open(os.path.join(folder_path, file), 'r') as f:
                try:
                    data = json.load(f)["response"]["data"]
                    electricity_data.extend(data)
                except KeyError:
                    print(f"Skipping file {file} due to unexpected format")
    electricity_df = pd.DataFrame(electricity_data)
    electricity_df["period"] = pd.to_datetime(electricity_df["period"], errors='coerce')
    electricity_df["value"] = electricity_df["value"].astype(str).str.extract(r'([0-9]+\.?[0-9]*)')[0]
    electricity_df["value"] = pd.to_numeric(electricity_df["value"], errors='coerce')
    electricity_df = electricity_df.rename(columns={"period": "datetime", "value": "demand_mwh"})
    electricity_df["datetime"] = pd.to_datetime(electricity_df["datetime"], errors='coerce', utc=True)
    return electricity_df.dropna()

def load_weather_data(folder_path):
    weather_data = []
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            df = pd.read_csv(os.path.join(folder_path, file))
            df.rename(columns=lambda x: x.strip(), inplace=True)
            df["date"] = pd.to_datetime(df["date"], errors='coerce', utc=True)
            weather_data.append(df)
    weather_df = pd.concat(weather_data, ignore_index=True)
    weather_df = weather_df.rename(columns={"date": "datetime", "temperature_2m": "temperature"})
    weather_df["datetime"] = pd.to_datetime(weather_df["datetime"], errors='coerce', utc=True)
    return weather_df.dropna()

def merge_data():
    # Load data
    electricity_df = load_electricity_data(electricity_data)
    weather_df = load_weather_data(weather_data)
    
    # Merge data
    data = pd.merge(electricity_df, weather_df, on="datetime", how="inner")
    data["demand_mwh"] = pd.to_numeric(data["demand_mwh"], errors='coerce')
    data.sort_values(by="datetime", inplace=True)

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Save merged data
    data.to_csv(output_file, index=False)
    
    return data


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import streamlit as st

def feature_engineering(data):
    
    # Feature Engineering
    data["hour"] = data["datetime"].dt.hour
    data["day"] = data["datetime"].dt.day
    data["month"] = data["datetime"].dt.month
    data['year'] = data["datetime"].dt.year
    data["day_of_week"] = data["datetime"].dt.dayofweek
    data["is_weekend"] = data["day_of_week"].apply(lambda x: 1 if x >= 5 else 0)
    data["season"] = data["month"].apply(lambda x: 'Winter' if x in [12, 1, 2] else 'Spring' if x in [3, 4, 5] else 'Summer' if x in [6, 7, 8] else 'Fall')
    data = pd.get_dummies(data, columns=['season'], drop_first=True)
    data = pd.get_dummies(data, columns=['subba-name'], drop_first=True)
    # Convert only the new dummy columns to int
    dummy_cols = [col for col in data.columns if 'Province_' in col]
    data[dummy_cols] = data[dummy_cols].astype(int)

    return data

def data_type_conversions(data):
    # Ensure demand_mwh is numeric
    data["demand_mwh"] = pd.to_numeric(data["demand_mwh"], errors='coerce')

    # Sort data by datetime
    data.sort_values(by="datetime", inplace=True)

    return data

def test_mcar(df):
    df_miss = df.isnull().astype(int)
    chi2, p, _, _ = chi2_contingency(df_miss.corr())
    return p  # If p > 0.05, data is MCAR

def data_cleaning_and_consistency(df):
    # 1. Identify missing values per column
    missing_values = df.isnull().sum()
    missing_percentage = (missing_values / len(df)) * 100

    # 2. Display missing values summary
    missing_summary = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})
    st.write("Missing Values Summary:")
    st.write(missing_summary[missing_summary['Missing Values'] > 0])
    
    p_value = test_mcar(df)
    if p_value > 0.05:
        st.write("Missing data is likely MCAR (Missing Completely at Random)")
    else:
        st.write("Missing data is likely MAR (Missing at Random) or MNAR (Not Missing at Random)")

    # 5. Handling missing data
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            if df[col].dtype == 'object':  # Categorical columns
                df[col].fillna(df[col].mode()[0], inplace=True)  # Impute with mode
            else:  # Numerical columns
                df[col].fillna(df[col].median(), inplace=True)  # Impute with median

    st.write("Missing values handled using mode (categorical) or median (numerical)")

    return df

def handle_duplicates_and_anomalies(df):
    # Remove duplicate rows
    df = df.drop_duplicates()

    # Identify numerical columns
    num_cols = df.select_dtypes(include=['number']).columns

    # Detect outliers using IQR method
    for col in num_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Mark outliers
        outliers = (df[col] < lower_bound) | (df[col] > upper_bound)
        st.write(f"Outliers detected in {col}: {outliers.sum()} rows")

    return df

def normalize_data(data):
    # Identify numerical columns for normalization/standardization
    num_cols = data.select_dtypes(include=['number']).columns

    # Choose either StandardScaler (Z-score normalization) or MinMaxScaler (scales to [0,1])
    scaler = StandardScaler()
    data[num_cols] = scaler.fit_transform(data[num_cols])
    return data

# def normalize_data(data):
#     # Normalize only the 'value' column
#     if 'value' in data.columns:
#         scaler = StandardScaler()
#         data['value'] = scaler.fit_transform(data[['value']])
#     return data


def process_data(df):
    df = data_type_conversions(df)
    df= data_cleaning_and_consistency(df)
    df = handle_duplicates_and_anomalies(df)
    df = feature_engineering(df)

    return df

SyntaxError: unterminated triple-quoted string literal (detected at line 34) (1691065487.py, line 6)

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

def statistical_summary(df):
    """
    Computes statistical metrics for each numerical variable in the dataframe and returns
    a prettified DataFrame.
    
    Parameters:
    df (pd.DataFrame): Input DataFrame with numerical variables
    
    Returns:
    pd.DataFrame: A DataFrame containing the statistical summaries
    """
    # Select numerical columns
    num_cols = df.select_dtypes(include=['number']).columns

    if len(num_cols) == 0:
        return "No numerical columns found in the DataFrame."

    # Dictionary to store summaries for each variable
    summary_dict = {}
    for col in num_cols:
            summary_dict[col] = {
                "Mean": df[col].mean(),
                "Median": df[col].median(),
                "Standard Deviation": df[col].std(),
                "Variance": df[col].var(),
                "Skewness": df[col].skew(),
                "Kurtosis": stats.kurtosis(df[col], fisher=True),
                "Min": df[col].min(),
                "Max": df[col].max(),
                "25th Percentile": df[col].quantile(0.25),
                "50th Percentile (Median)": df[col].quantile(0.50),
                "75th Percentile": df[col].quantile(0.75)
            }

        # Convert dictionary to DataFrame for pretty display
    summary_df = pd.DataFrame(summary_dict).T
    
    return summary_df

def plot_time_series(df, datetime_col="datetime", demand_col="demand_mwh", sample_rate=1000):
    """
    Plots a clean time series line chart without unwanted fill effects.

    :param df: Pandas DataFrame containing time series data
    :param datetime_col: Column name representing timestamps
    :param demand_col: Column name representing electricity demand
    :param sample_rate: Interval to downsample data for better visualization
    """
     # Convert datetime column to proper format
    df[datetime_col] = pd.to_datetime(df[datetime_col])

    # Sort data by time
    df = df.sort_values(by=datetime_col)

    # Downsample data (optional but improves clarity)
    # df_sampled = df.iloc[::sample_rate]  # Pick every nth row
    df_sampled = df

    # Set figure size
    plt.figure(figsize=(14, 6))

    # Plot as a thin line (no fill)
    plt.plot(df_sampled[datetime_col], df_sampled[demand_col], color='blue', linewidth=1, linestyle='-')

    # Titles and labels
    plt.title("Electricity Demand Over Time", fontsize=14)
    plt.xlabel("Time", fontsize=12)
    plt.ylabel("Electricity Demand", fontsize=12)

    # Rotate x-axis labels for better visibility
    plt.xticks(rotation=45)

    # Grid for better visualization
    plt.grid(True, linestyle="--", alpha=0.6)

    # Show plot
    plt.show()

def univariate_analysis(df):
    """
    Perform univariate analysis: Histogram, Boxplot, Density plot.
    
    :param df: Pandas DataFrame containing the data
    :param column: Column name (numerical feature) to analyze
    """
      # Convert datetime column to actual datetime format
    df["datetime"] = pd.to_datetime(df["datetime"])

    # Selecting only numerical columns
    numerical_cols = df.select_dtypes(include=["number"]).columns

    # Iterate over each numerical column
    for col in numerical_cols:
        plt.figure(figsize=(18, 5))

        # Histogram
        plt.subplot(1, 3, 1)
        sns.histplot(df[col], bins=30, kde=True, color='blue')
        plt.title(f"Histogram of {col}")

        # Boxplot
        plt.subplot(1, 3, 2)
        sns.boxplot(y=df[col], color='green')
        plt.title(f"Boxplot of {col}")

        # Density Plot
        plt.subplot(1, 3, 3)
        sns.kdeplot(df[col], fill=True, color='red')
        plt.title(f"Density Plot of {col}")

        plt.show()

        # Statistical Summary
        stats = df[col].describe()
        skewness = df[col].skew()
        kurtosis = df[col].kurtosis()

        print(f"\n🔹 Statistical Summary for {col}:\n")
        print(stats)
        print(f"Skewness: {skewness}")
        print(f"Kurtosis: {kurtosis}\n")
        print("-" * 50)

def correlation_analysis(df, threshold=0.75):
    """
    Computes and visualizes the correlation matrix for numerical features.
    Identifies multicollinearity issues by flagging highly correlated features.

    Parameters:
    df (pd.DataFrame): Input DataFrame
    threshold (float): Correlation threshold for identifying multicollinearity (default = 0.75)

    Returns:
    high_corr_pairs (list): List of highly correlated feature pairs
    """
    # Select numerical columns
    numerical_cols = df.select_dtypes(include=['number']).columns

    # Compute correlation matrix
    corr_matrix = df[numerical_cols].corr()

    # Plot heatmap
    plt.figure(figsize=(12, 6))
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
    plt.title("Correlation Matrix Heatmap")
    plt.show()

    # Identifying Multicollinearity (correlation > threshold)
    high_corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))
  # Print highly correlated features
    if high_corr_pairs:
        print("\n🔹 Highly Correlated Feature Pairs (|correlation| > {}):".format(threshold))
        for feature1, feature2, correlation in high_corr_pairs:
            print(f"{feature1} ↔ {feature2} | Correlation: {correlation:.2f}")
    else:
        print("\n✅ No strong multicollinearity detected (correlation > {}).".format(threshold))
    
    return high_corr_pairs

def time_series_analysis(df, date_col, target_col, period=24):
    """
    Performs time series decomposition and stationarity testing (ADF Test).

    Parameters:
    df (pd.DataFrame): Input DataFrame with time series data
    date_col (str): Column name containing datetime information
    target_col (str): Column name for the time series variable (e.g., demand)
    period (int): Seasonal period for decomposition (default = 24 for hourly data)

    Returns:
    None
    """
    # Convert date column to datetime
    df[date_col] = pd.to_datetime(df[date_col])
    df.set_index(date_col, inplace=True)

    # Time Series Decomposition
    decomposition = sm.tsa.seasonal_decompose(df[target_col], model='additive', period=period)

    # Plot decomposition
    fig, axes = plt.subplots(4, 1, figsize=(12, 8))
    decomposition.observed.plot(ax=axes[0], title="Observed")
    decomposition.trend.plot(ax=axes[1], title="Trend")
    decomposition.seasonal.plot(ax=axes[2], title="Seasonality")
    decomposition.resid.plot(ax=axes[3], title="Residuals", linestyle='dashed')
    for ax in axes:
        ax.set_xlabel("Date")
    
    plt.tight_layout()
    plt.show()

    # Stationarity Test: Augmented Dickey-Fuller Test
    print("\n📉 Augmented Dickey-Fuller Test Results:")
    adf_test = adfuller(df[target_col].dropna())
    results = pd.Series(adf_test[:4], index=['Test Statistic', 'p-value', '# Lags Used', '# Observations Used'])
    for key, value in adf_test[4].items():
        results[f'Critical Value ({key})'] = value

    print(results)

    # Interpretation
    if adf_test[1] < 0.05:
        print("\n✅ The time series is stationary (p-value < 0.05).")
    else:
        print("\n⚠️ The time series is non-stationary (p-value >= 0.05). Consider differencing or detrending.")

    # Reset index after analysis
    df.reset_index(inplace=True)

def check_demand_in_each_city(data):
    # Extract province columns (all columns starting with "Province_")
    province_columns = [col for col in data.columns if col.startswith("Province_")]
    
    # Multiply each one-hot encoded province column by the demand to distribute demand correctly
    province_demand = data[province_columns].multiply(data["demand_mwh"], axis=0).sum()

    # Sort values
    province_demand = province_demand.sort_values()

    # Plot
    plt.figure(figsize=(12, 6))
    sns.barplot(y=province_demand.index.str.replace("Province_", ""), x=province_demand.values, palette="coolwarm", orient='h')
    plt.ylabel("Province")
    plt.xlabel("Total Electricity Demand (MWh)")
    plt.title("Electricity Demand by Province")
    plt.show()

    return province_demand

def perform_eda(df):

    check_demand_in_each_city(df)

    statistical_summary_result = statistical_summary(df)
    print(statistical_summary_result)

    plot_time_series(df)

    univariate_analysis(df)

    high_corr_pairs = correlation_analysis(df)

    time_series_analysis(df, "datetime", "demand_mwh")

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = (data[column] < lower_bound) | (data[column] > upper_bound)
    return outliers

def detect_outliers_zscore(data, column, threshold=3):
    z_scores = np.abs(zscore(data[column]))
    return z_scores > threshold

def detect_and_handle_outliers(df, column, method="remove"):
    """
    Detects and handles outliers in the specified column of the dataset.

    Parameters:
        df (pd.DataFrame): The dataset containing the column.
        column (str): The column name to analyze.
        method (str): Strategy to handle outliers - "remove", "cap", or "transform".

    Returns:
        pd.DataFrame: The dataset after applying the outlier handling strategy.
    """

    # Handle Outliers Based on the Selected Method
    df_cleaned = df.copy()
    
    if method == "remove":
        iqr_outliers = detect_outliers_iqr(df, column)
        df_cleaned = df_cleaned[~iqr_outliers]  # Remove outliers
    elif method == "cap":
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_cleaned[column] = np.clip(df_cleaned[column], lower_bound, upper_bound)  # Cap values
    elif method == "transform":
        df_cleaned[column] = np.log1p(df_cleaned[column])  # Log transformation

    # Plot Before and After
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    
    axs[0].hist(df[column], bins=50, color="blue", alpha=0.7)
    axs[0].set_title(f"Original {column} Distribution")

    axs[1].hist(df_cleaned[column], bins=50, color="red", alpha=0.7)
    axs[1].set_title(f"After Applying {method.capitalize()} Method")

    plt.show()
    
    return df_cleaned

def outliers(df):

    # Apply to demand_mwh column
    iqr_outliers = detect_outliers_iqr(df, "demand_mwh")
    print("IQR Outliers (demand_mwh):", iqr_outliers.sum())
    zscore_outliers = detect_outliers_zscore(df, "demand_mwh")
    print("Z-score Outliers (demand_mwh):", zscore_outliers.sum())
    
    # Apply to temperature column
    iqr_outliers = detect_outliers_iqr(df, "temperature")
    print("IQR Outliers (temperature):", iqr_outliers.sum())
    zscore_outliers = detect_outliers_zscore(df, "temperature")
    print("Z-score Outliers (temperature):", zscore_outliers.sum())

    # Run Outlier Detection & Handling
    cleaned_data = detect_and_handle_outliers(df, "demand_mwh", method="transform")
    cleaned_data = detect_and_handle_outliers(cleaned_data, "temperature", method="remove")

    return cleaned_data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

def regression_model(df, target, time_column, test_size=0.2):
    """
    Builds and evaluates a regression model to predict electricity demand.

    Parameters:
        df (pd.DataFrame): The preprocessed dataset.
        target (str): The column name of the target variable (e.g., electricity demand).
        time_column (str): The column representing timestamps.
        test_size (float): Proportion of data used for testing (default = 0.2).

    Returns:
        model (LinearRegression): Trained regression model.
        predictions (np.array): Predictions on test data.
    """

    # Define Features (Excluding Non-Numeric Columns)
    feature_cols = ["hour", "day", "month", "year", "day_of_week", "is_weekend", "season_Spring", "season_Summer", "season_Winter"]
    if "temperature" in df.columns:
        feature_cols.append("temperature")  # Include temperature if available

    province_columns = [col for col in df.columns if col.startswith("Province_")]
    feature_cols.extend(province_columns)

    X = df[feature_cols]
    y = df[target]

    # Split into Training & Testing Sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # Train Linear Regression Model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predictions
    y_pred = model.predict(X_test)

    # Evaluate Model
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print("Model Performance Metrics:")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R² Score: {r2:.2f}")
    print(f"Model Score: {model.score(X_test, y_test):.2f}")

    # Plot Actual vs Predicted
    plt.figure(figsize=(10, 5))
    plt.scatter(y_test, y_pred, alpha=0.5, color="blue")
    plt.plot([y.min(), y.max()], [y.min(), y.max()], '--r', linewidth=2)  # Perfect fit line
    plt.xlabel("Actual Demand")
    plt.ylabel("Predicted Demand")
    plt.title("Actual vs Predicted Electricity Demand")
    plt.show()

    # Residual Plot
    residuals = y_test - y_pred
    plt.figure(figsize=(10, 5))
    sns.histplot(residuals, bins=30, kde=True, color="purple")
    plt.axvline(0, color='red', linestyle='dashed', linewidth=2)
    plt.xlabel("Residuals")
    plt.title("Residual Analysis")
    plt.show()

    return model, y_pred

In [7]:
from eda import perform_eda
from loader import merge_data
from processor import process_data, normalize_data
from outlier import outliers
from regression import regression_model

if __name__ == "__main__":
    print("Exploratory Data Analysis and Regression Model")

    # Data Loading
    print("Data Loading")
    raw_df = merge_data()
    raw_df.to_csv("raw_merged_data.csv", index=False)
    print("Raw data loaded and saved as 'raw_merged_data.csv'.")

    # Data Processing
    print("Data Processing")
    processed_df = process_data(raw_df)
    processed_df.to_csv("processed_data.csv", index=False)
    print("Processed data saved as 'processed_data.csv'.")

    # EDA
    print("Exploratory Data Analysis (EDA)")
    perform_eda(processed_df)
    print("EDA performed on processed data.")

    # Outlier Detection
    print("Outlier Detection")
    cleaned_df = outliers(processed_df)
    cleaned_df.to_csv("cleaned_data.csv", index=False)
    print("Outliers detected and cleaned data saved as 'cleaned_data.csv'.")

    # Normalization
    print("Normalization")
    normalized_df = normalize_data(cleaned_df)
    normalized_df.to_csv("normalized_data.csv", index=False)
    print("Normalized data saved as 'normalized_data.csv'.")

    # Regression Model
    print("Regression Model")
    model, predictions = regression_model(normalized_df, target="demand_mwh", time_column="datetime")
    print("Regression model built and evaluated.")

Exploratory Data Analysis and Regression Model
Data Loading


ParserError: Error tokenizing data. C error: out of memory

In [5]:
    # Regression Model
    print("Regression Model")
    model, predictions = regression_model(normalized_df, target="demand_mwh", time_column="datetime")
    print("Regression model built and evaluated.")

Regression Model


NameError: name 'normalized_df' is not defined